# Talk like the President, part 2

Welcome back!

In this part we will continue to make changes to our model. Hopefully we will end up with a model that can take much longer sequences into account and that is also easier to train.

Additionally, in order to improve the quality of generated texts we will try our hand at beam search.

Without further ado, let's get started!

In [1]:
import glob
from utils import *

from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

In [2]:
# You should already have the speeches downloaded. If not, please run the code in part 1

# Loading in speeches and doing the usual maintance work
speeches = ' '.join(preprocess_speech(file) for file in glob('data/Corpus of Presential Speeches/obama/*'))

chars = sorted(list(set(speeches)))
vocab_size = len(chars)
print('total chars:', vocab_size) # Sanity check 1

char2idx = {c: idx for idx, c in enumerate(chars)}
idx2char = {idx: c for idx, c in enumerate(chars)}
idxs = [char2idx[char] for char in speeches]
print(''.join(idx2char[idxs[i]] for i in range(100))) # Sanity check 2

total chars: 74
Mr. Speaker, Mr. Vice President, members of Congress, distinguished guests, and fellow Americans: La


## Multi-output model

So far we have been running the model on some amount of chars and predicting the next one. Then we repeat the process - we run the model on n characters again and predict the next one.

This seems to be wasting a lot of opportunities to train, especially with longer sequences. 

Could we also predic the intermediate chars in the sequence and train on those? See 1st char, predict 2nd char, see 2nd char, predict 3rd char, and so on.

Let's start with creating the dataset.

In [3]:
c_dat = [idxs[i:i+8] for i in range(len(idxs)-8)]

In [4]:
c_in_dat = c_dat[:-1]
c_out_dat = c_dat[1:]

In [5]:
# Checking if everything looks okay
c_in_dat[:5], c_out_dat[:5]

([[35, 65, 7, 0, 41, 63, 52, 48],
  [65, 7, 0, 41, 63, 52, 48, 58],
  [7, 0, 41, 63, 52, 48, 58, 52],
  [0, 41, 63, 52, 48, 58, 52, 65],
  [41, 63, 52, 48, 58, 52, 65, 5]],
 [[65, 7, 0, 41, 63, 52, 48, 58],
  [7, 0, 41, 63, 52, 48, 58, 52],
  [0, 41, 63, 52, 48, 58, 52, 65],
  [41, 63, 52, 48, 58, 52, 65, 5],
  [63, 52, 48, 58, 52, 65, 5, 0]])

In [6]:
xs = np.stack(c_in_dat)
ys = np.stack(c_out_dat)
xs.shape, ys.shape

((1027432, 8), (1027432, 8))

In [7]:
# This one line allows me to run my models on a subset of data while I work on this notebook
md = ColumnarModelData.from_arrays('.', [-20_000], xs[:200_000, :], ys[:200_000], bs=2**9)

# md = ColumnarModelData.from_arrays('.', [-100_000], xs, ys, bs=2**9)

#### Creating the model

In [8]:
n_fac = 42 # number of latent factors, embedding length
n_hidden = 256

In [9]:
class CharSeqRNN(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h0 = V(torch.zeros(1, bs, n_hidden).cuda())
        inp = self.e(torch.stack(cs))
        output, hn = self.rnn(inp, h0)
        return F.log_softmax(self.l_out(output), dim=-1)

In [10]:
m = CharSeqRNN(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

Unfortunately, at the time of writing the published pytorch version is still 0.3. That means that the multidimensional nll_loss has not made it into a release yet.

Because we will now be outputting a prediction for 8 chars, instead of one, we need to write a wrapper around the pytorch's nll_loss as it only works with outputs of shape (N, C).

In [11]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [12]:
fit(m, md, 1, opt, nll_loss_seq)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.70905  1.93887]                                



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.58125  1.7642 ]                                



To produce text we can still use nearly the same get_next method as we were using in the past (we just need to make sure we pass it the last prediction our model outputs vs predictions after seeing each char in the sequence)

In [13]:
def get_next(inp):
    idxs = T(np.array([char2idx[c] for c in inp]))
    p = m(*VV(idxs))[-1].exp()
    r = torch.multinomial(p, 1)
    return idx2char[r.data[0, 0]]

But this time we want to be feeding it a sequence of length <= 8

In [14]:
text = 'Hello'
for i in range(200): text += get_next(text[(-8 if len(text) > 7 else 0):])

print(text)

Helloy. Aftart tod businession, wholingly hather remidam Mip. Our peacemer in their deficit to we must to America know their time that is mrlar, thry hese prograf immiare Areamber abrors alreeps that may g


It works, though not great, as expected.

One minor trick that we can use to help with training is initializing the hidden state to an identity matrix. (I think this should generally be helpful though mileage might vary - [best used with ReLu](https://arxiv.org/abs/1504.00941) and we are using tanh here)

In [15]:
m = CharSeqRNN(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [16]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [17]:
fit(m, md, 1, opt, nll_loss_seq)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.75226  2.50406]                                



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.62457  2.15972]                                



Time to make another addition to our toolbox - making the RNN stateful. If you would like watch Jeremy explain this in detail, this is covered in [lecture #7](https://youtu.be/H3g26EVADgY?t=6m13s).

## Stateful RNN

Instead of discarding the hidden state after each batch we will retain it to have a better starting point for the next batch. This will requires our dataset to be constructed in a very specific way. This also means that we have to somehow preventing backpropagating the error through multiple batches (otherwise we will get a NN with thousands of layers and that is not what we want).

Instead of writing our own dataset machinery, let's employ torchtext.

In [18]:
from torchtext import vocab, data

# if you have not used spacy before you might need to download the english model
# via executing the below from terminal
#   python -m spacy download en
from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

!mkdir -p {TRN}
!mkdir -p {VAL}

with open(f'{TRN}trn.txt', 'w') as text_file:
    text_file.write(speeches[:800_000])
    
with open(f'{VAL}val.txt', 'w') as text_file:
    text_file.write(speeches[800_000:])

%ls {PATH}

Corpus of Presential Speeches/  corpus.zip  models/  trn/  val/


Loading data using the functionality provided by fastai and torchtext

In [19]:
TEXT = data.Field(lower=False, tokenize=list)
bs=128; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(780, 75, 1, 800001)

Well, maybe this worked, maybe it didn't :) Let's see what we get


In [20]:
it = iter(md.trn_dl)
batch = next(it)
len(batch), batch[0].shape, batch[1].shape

(2, torch.Size([11, 128]), torch.Size([1408]))

Ok, so this is nice. The number of characters in a batch will slightly vary and also seems our targets are unrolled into a single dimensional vector.

In [21]:
class CharSeqStatefulRNN(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size # we will need it to modify the output of our RNN to work with F.nll_loss
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs) # this is new - we will initialize the hidden state once, at model creation vs
                             # reinitializing it at the beginning of ever batch
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs) # in case our last batch contains a lesser number of rows
        inp = self.e(cs)
        output, hn = self.rnn(inp, self.h)
        self.h = repackage_var(hn) # repackaging a var is a fastai function for h to forget history
        return F.log_softmax(self.l_out(output), dim=-1).view(-1, self.vocab_size) # this will allow us to use F.nll_loss

    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [22]:
m = CharSeqStatefulRNN(md.nt, n_fac, bs).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [23]:
fit(m, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.68157  1.71193]                                 



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.43695  1.44754]                                 



As our model now expexts an input of shape (seq_len, batch, input_size), we need to modify our `get_next` function.

In [24]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))[-1].exp()
    r = torch.multinomial(p, 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [25]:
text = 'Hello'
for i in range(200): text += get_next(text[(-8 if len(text) > 7 else 0):])

print(text)

Hellow, those violence. So faition-was an corports work diden with tax make quillyy passnd issue just years detent that next papreas. Let this drities, even the fad this purstition and scould is un righem 


In the [lecture video](https://youtu.be/H3g26EVADgY?t=47m19s) Jeremy provides a great explanation of how the GRU and LSTM cells work.

There are also great tutorials available. I learned about them from the wiki for lesson #7 on the [fast.ai forums](http://forums.fast.ai/t/wiki-lesson-7/9405).

I reference them here for your convenience:

* [Chris Olah on LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [WILD ML RNN Tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)

I will not go into the details of how the cells work. If you are interested the materials above are the best source to study this further.

The good news is that we already know everything we need to start using either the GRU or the LSTM cell. From outside they both are very similar to the nn.RNN layer.

There are two things that still remain. Fully training the model using one of the more complex cells and implementing beam search.

Let's start with constructing and training the model.

## Training the LSTM model

First of all, we the data. This time around, let's lower the batch size but increase the bptt significantly.

Increasing the bptt will allow our RNN to learn from longer sequences.

In [26]:
TEXT = data.Field(lower=False, tokenize=list)
bs=64; bptt=72; n_fac=42;

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(172, 75, 1, 800001)

In [27]:
n_hidden = 512 # Let's make our hidden state slightly bigger.

In [28]:
class Char_LSTM_RNN(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size = vocab_size
        self.nl = nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs) # in case our last batch contains a lesser number of rows
        inp = self.e(cs)
        output, hn_and_cell_state = self.rnn(inp, self.h)
        self.h = repackage_var(hn_and_cell_state) # repackaging a var works with a tuple as well!
        return F.log_softmax(self.l_out(output), dim=-1).view(-1, self.vocab_size) # this will allow us to use F.nll_loss

    def init_hidden(self, bs): 
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden))) # we need this now because LSTM expects to be given both
                                                   # the initial hidden state but also the cell state

Because we are using the fastai library, we have access to all of its goodies. This means we are just a few keystrokes away from using the wonderful cosine annealing of the learning rate!

And we can also save the intermediate model at cycle ends. We could probably further improve perofrmance via ensembling a couple of the best models.

In [29]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [30]:
m = Char_LSTM_RNN(md.nt, n_fac, bs, 3).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [31]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**5-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

[ 0.       2.68748  2.53559]                                
[ 1.       2.03807  1.80079]                                
[ 2.       1.70679  1.61632]                                
[ 3.       1.58691  1.46066]                                
[ 4.       1.44753  1.34418]                                
[ 5.       1.35991  1.28386]                                
[ 6.       1.3099   1.26304]                                
[ 7.       1.41269  1.32544]                                
[ 8.       1.37102  1.29049]                                
[ 9.       1.32688  1.25503]                                
[ 10.        1.28493   1.22384]                             
[ 11.        1.24576   1.19582]                             
[ 12.        1.20741   1.1722 ]                             
[ 13.        1.17764   1.1587 ]                             
[ 14.        1.16298   1.15617]                             
[ 15.        1.31912   1.25402]                             
[ 16.        1.30095   1

We have now trained our model. We could explore the architecture a bit more (add hiden layers, change embedding size, stack LSTMs) but the defaults taken from the [fastai lesson 6 notebook](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson6-rnn.ipynb) work quite well.

## Beam Search

If you would like to learn more about ways to do search (depth / breadth first search, hill climbing, beam search), this is a great [lecture to watch](https://www.youtube.com/watch?v=j1H3jAAGlEA)

This [2 minute video](https://www.youtube.com/watch?v=UXW6Cs82UKo) from Udacity talks about beam search only.

I will implement the beam search using basic data structures only, such as lists and tuples.

But first, let's modify the `get_next` function so that it returns both the predicted character and its probability.

In [32]:
def get_next(inp):
    '''Return predicted char and the probability of prediction'''
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))[-1].exp()
    r = torch.multinomial(p, 1)
    return TEXT.vocab.itos[to_np(r)[0]], p[r].data.cpu()[0]

We can start with a breadth first search. Going from it to beam search is trivial.

Breadth first search would allow us ot explore the entire solution space. But at a cost. If we were to generate a string of length 10 trying out each possible combination of characters, that would give us 75^10 = 5631351470947265625 possible solutions!

In [33]:
def breadth_first_search(texts, steps):
    '''
    Args:
        texts: a list of tuples of the form (<text>, <probability>)
        steps: steps to go
    '''
    if steps == 0: return texts
    
    new_texts = []
    
    for text, prob_text in texts:
        dist = get_distribution(text)
        for char, prob_char in dist:
            new_texts.append((text + char, prob_text + prob_char))
    
    new_texts = sorted(new_texts, key=lambda tup: tup[1], reverse=True)
    
    return breadth_first_search(new_texts, steps - 1)

def get_distribution(inp):
    idxs = np.array([TEXT.vocab.stoi[c] for c in inp])
    idxs = idxs.reshape(-1, 1) # RNNs in PyTorch expect input of dim [seq_len x batch_size x embedding_size]
    p = m(VV(idxs))[-1]
    chars_with_prob = zip(TEXT.vocab.itos, p.data.cpu().numpy())
    return sorted(chars_with_prob, key=lambda tup: tup[1], reverse=True)

In [34]:
%%time
result = breadth_first_search([('I am', 0)], 3)
print(f'Found {len(result)} 3 char combinations')
result[:5]

Found 421875 3 char combinations
CPU times: user 6.09 s, sys: 2.2 s, total: 8.29 s
Wall time: 8.29 s


To the best of my understanding, the implementation of beam search below is correct. It unfortunately was not giving great results - the outputs were too conservative. The serach was producing text that had high probability according to the RNN but with a lot repetitions.

I came up with a `stochastic beam search` where the results are better.

In [35]:
def beam_search(texts, steps, beam_size):
    '''
    Args:
        texts: a list of tuples of the form (<text>, <probability>)
        steps: steps to go
        beam_size: count of candidate branches to keep
    '''
    if steps == 0: return texts
    
    new_texts = []
    
    for text, prob_text in texts:
        dist = get_distribution(text)
        for char, prob_char in dist:
            new_texts.append((text + char, prob_text + prob_char))
    
    new_texts = sorted(new_texts, key=lambda tup: tup[1], reverse=True)[:beam_size]
    
    return beam_search(new_texts, steps - 1, beam_size)

In [36]:
def stochastic_beam_search(texts, steps, beam_size, candidates_to_evaluate):
    '''
    Args:
        texts: a list of tuples of the form (<text>, <log_probability>)
        steps: steps to go
        beam_size: count of candidate branches to keep
        candidates_to_evaluate: how many times should we expand a given branch at each step
    '''
    if steps == 0: return texts[0][0]
    new_texts = []
    
    for text, prob_text in texts:
        for _ in range(candidates_to_evaluate):
            c, p = get_next(text)
            new_texts.append((text + c, prob_text + np.log(p)))
        
    new_texts = sorted(new_texts, key=lambda tup: tup[1], reverse=True)[:beam_size]
    return stochastic_beam_search(new_texts, steps - 1, beam_size, candidates_to_evaluate)

In [37]:
%%time
stochastic_beam_search([('The meaning of life is ', 0)], 400, 1, 2)

CPU times: user 9.29 s, sys: 1.31 s, total: 10.6 s
Wall time: 10.6 s


"The meaning of life is that we are a better strength of college than the law in our own, and the world of the depends of a decade of interests who are a single lives of a training and the future that despite its ideas. And they don't take the country in the world's faith to our economy. We will not threaten a solve the facts of the same possible challenges that the community is a safe haven in the more than the same res"

And here we repeat the above but with beam of size 1, which in our case equates to sampling directly from the softmax.

In [38]:
%%time
stochastic_beam_search([('The meaning of life is ', 0)], 400, 1, 1)

CPU times: user 4.5 s, sys: 804 ms, total: 5.3 s
Wall time: 5.3 s


"The meaning of life is both pointing, when we worked without extremists for me set answer and everyone faced to do onting idea up nuclear very doubts how because we will change the weeks. We is containing together. We need so much family by the history. So America needs to be areay with issues, no coal that makes businesss. We'll defend our applying prayers and good with worker or a char-off religion to ockane, what wou"

Probably the stochastic beam search (a name I made up) would be the winner but hard to say. One thing that might be interesting to explore would be prunning not after n + 1 chars, but after n + i for some i > 1. This might allow for greater variety of branches that make it into the beam. We might also be able to prune branches that turn out bad only further downstream.

As generating sequences is relatively fast, one could also consider generating some number of sequences sampling directly from the softmax and prunning afterwards. For example, we could generate 20 strings and pick the one that the RNN believest to have greates probability.

## Where to go from here

I would definitely recommend reading an article by Andrej Karpathy on [the unreasonable effectiveness of RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) that has now become a classic.

With basic understanding of RNNs and the inner workings of the GRU and the LSTM cells, the world is an oyster. Really.

There are so many architectures to explore. Bi-directional RNNs, seq2seq, attention models. You might jump into exploring the field on your own or if you prefer having an experienced guide, I would recommend checking out the [Cutting Edge Deep Learning for Coders](http://course.fast.ai/part2.html) by [fastai](http://www.fast.ai/).

And this is it! I do not envision there being a part 3, but the list of things I would like to work on and potentially write about is at this point endless :) See you on [twitter](https://twitter.com/radekosmulski) or on the [forums](http://forums.fast.ai/)!